# ICMS Maintenance

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
icms1 = pd.read_csv('/Users/ryanbreier/RB.Data & Programming/Data Science/Testing/icms_maintenance/icms_export_1.csv')
icms2 = pd.read_csv('/Users/ryanbreier/RB.Data & Programming/Data Science/Testing/icms_maintenance/icms_export_2.csv')

#### Inspect dataframe

In [ ]:
# icms1.head()
# col_list = icms1.columns.tolist()
# col_list

# icms1.dtypes
# icms1.describe()
# icms1.shape # (row_count, column_count)

# icms1.shape[0] ## call rows_count
# icms1.shape[1] ## call columns_count

#### Creat UniqueID field

In [ ]:
icms1['UniqueID'] = icms1['DataProvider'] + '//' + icms1['BaseAgreementID'].astype(str) + '//' + \
icms1['CommercialName'] + '//' + icms1['ProductName'] + '//' + icms1['SplitNum'].astype(str)

icms2['UniqueID'] = icms2['DataProvider'] + '//' + icms2['BaseAgreementID'].astype(str) + '//' + \
icms2['CommercialName'] + '//' + icms2['ProductName'] + '//' + icms2['SplitNum'].astype(str)

## Merge versions

In [ ]:
# icms2 = icms2[['UniqueID','AcctAssignGL','AcctAssignCC','TermDate','Amount']]

# Merge DataFrames
icms_merged_inner = icms1.merge(icms2,how='inner',on=['UniqueID'],suffixes=('_old','_new'))
icms_merged_outer = icms1.merge(icms2,how='outer',on=['UniqueID'],suffixes=('_old','_new'))
# icms_merged_inner

#### CC changes

In [ ]:
icms_changed_cc = icms_merged_inner[icms_merged_inner['AcctAssignCC_old'] != icms_merged_inner['AcctAssignCC_new']]
icms_changed_cc[['DataProvider_new','CommercialName_new','BU_new','AcctAssignCC_old','AcctAssignCC_new']]

#### G/L changes

In [ ]:
icms_changed_gl = icms_merged_inner[icms_merged_inner['AcctAssignGL_old'] != icms_merged_inner['AcctAssignGL_new']]
icms_changed_gl[['DataProvider_new','CommercialName_new','BU_new','AcctAssignGL_old','AcctAssignGL_new']]

#### Annual Amount changes

In [ ]:
icms_changed_amt = icms_merged_inner[icms_merged_inner['Amount_old'] != icms_merged_inner['Amount_new']]
icms_changed_amt = icms_changed_amt[['DataProvider_new','CommercialName_new','BU_new','Amount_old','Amount_new']]

icms_changed_amt['Change'] = icms_changed_amt['Amount_new'] - icms_changed_amt['Amount_old']
icms_changed_amt

#### New Contracts

In [ ]:
new_icms = icms_merged_outer[icms_merged_outer['BaseAgreementID_old'].isnull()]
new_icms = new_icms.rename(columns={'DataProvider_new':'DataProvider',
                                    'BaseAgreementID_new':'BaseAgreementID'
                                   })
new_icms[['DataProvider','BaseAgreementID','BU_new','AcctAssignGL_new','AcctAssignCC_new']]

#### Removed Contracts

In [ ]:
rem_icms = icms_merged_outer[icms_merged_outer['BaseAgreementID_new'].isnull()]
rem_icms = rem_icms.rename(columns={'DataProvider_old':'DataProvider',
                                    'BaseAgreementID_old':'BaseAgreementID'
                                   })
rem_icms[['DataProvider','BaseAgreementID','BU_old','AcctAssignGL_old','AcctAssignCC_old','Amount_old']]

## Examples

### Indexing

#### Index DataFrame by column name

In [ ]:
# icms1[['DataProvider','BaseAgreementID','CommercialName']].head(3)

#### .loc/.iloc --> Access a group of rows and columns by label(s) or a boolean array: [ r , c ]

In [ ]:
# icms1.loc[:,'DataProvider'].head(3)
# icms1.iloc[:,0:5].head(5)

#### value_counts --> counts distinct # of values in a given series

In [ ]:
# icms2['DataProvider'].value_counts()
# pd.value_counts(icms1['DataProvider'])

#### sort_values --> sort by column(s) in a given data frame

In [ ]:
# icms1_sorted = icms1.sort_values(by=['DataProvider','TermDate'],ascending=False)
# icms1_sorted.head()

# icms2_sorted = icms2.sort_values(by=['Amount'], ascending=False)
# icms2_sorted.head(3)

#### Group By (Data Provider) and Aggregate (Amount)

In [ ]:
# icms1.groupby(['DataProvider']).agg({'Amount':'count'})
# icms1.groupby(['DataProvider']).agg({'Amount':'mean'})
# icms1.groupby(['DataProvider']).agg({'Amount':['min','max','mean']})
# icms1.groupby(['DataProvider','CommercialName']).agg({'Amount':['min','max','mean']})

Filter dataframe on certain value(s) within certain column(s)

In [ ]:
# icms1[icms1['DataProvider']=='AAA']
# icms1[(icms1['BaseAgreementID']==111) | (icms1['TermDate']<='12/10/2019')].head(3)

Count the number of base agreements for Data Providers names containing 'AAA'

In [ ]:
# icms1_AAA = icms1[icms1['DataProvider'].str.contains('AAA')]
# icms1_AAA['BaseAgreementID'].value_counts()

## Groupby

In [ ]:
# list(oo.groupby('Edition'))
# group_by = 'Edition'
# oo_ath = oo.groupby(group_by)

In [ ]:
# for group_key, group_value in oo_ath:
#     print(group_key)
#     print(group_value)

#### Groupby Computations

In [ ]:
# oo_ath.size().head()

In [ ]:
# oo[oo['Athlete'] == 'LEWIS, Carl'].groupby(['Athlete','Medal','Event']).agg({'Edition' :['min','max','count']})

#### Create a list showing the total number of medals won for each country over the history of the Olympics. For each country, include the year of the first and the most recent Olympic medal wins.

In [ ]:
# oo.groupby('NOC').agg({'Edition' :['min','max','count']}).head()

### Reshaping

In [ ]:
# mw = oo[(oo['Edition'] == 2008) & ((oo['Event'] == '100m') | (oo['Event'] == '200m'))]
# mw.head()

In [ ]:
# g = mw.groupby(['NOC','Gender','Discipline','Event']).size()
# g

##### Unstack --> move series' (Discipline & Event) to columns

In [ ]:
# g.unstack(['Discipline','Event'], fill_value=0)

## Basic Plotting

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# icms2['BU'].value_counts().plot(kind='hist',title='# of Contracts by Business');
# icms2['BU'].value_counts().plot(kind='bar',color='b',title='# of Contracts by Business');
# icms2['BU'].value_counts().plot(kind='barh',color='b',title='# of Contracts by Business');
# icms2['BU'].value_counts().plot(kind='pie',colors='grb',legend=True,title='# of Contracts by Business');

## Seaborn

In [ ]:
import seaborn as sns
# flatui = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
# palplot_fui = sns.palplot(sns.color_palette(flatui))

#### Product Counts by Business

In [ ]:
# sns.countplot(x='ProductName',data=icms2,hue='BU',palette=palplot_fui);

#### # of Contracts by Business

In [ ]:
# china2008 = oo[(oo['Edition'] == 2008) & (oo['NOC'] == 'CHN')]
# sns.countplot(x='BU', data=icms2, palette=palplot_fui);

###### Plot the number Gold, Silver, and Bronze medals for each gender (China/2008 dataset)

In [ ]:
# sns.countplot(x='Medal', data=china2008, palette='bwr', hue='Gender', order=['Gold', 'Silver', 'Bronze']);

#### Using groupby, plot the total number of medals awarded at each of the Olympic games throughout history 

In [ ]:
# oo_grouped = oo.groupby('Edition').size()
# oo_grouped = oo.groupby('Edition').agg({'Medal':'count'})
# oo_grouped.plot();

#### Plot the number of gold medals won by the USA male and female athletes throughout the history of the Olympics.

In [ ]:
# usa_gold = oo[(oo['NOC'] == 'USA') & (oo['Medal'] == 'Gold')]
# usa_gold_mw = usa_gold.groupby(['Edition', 'Gender']).size()
# usa_gold_mw.unstack('Gender', fill_value=0).plot();
# type(usa_gold)

#### Plot the five athletes who have won the most gold medals over the history of the Olympics. When there is a tie, consider the number of silver medals and then bronze medals.

In [ ]:
# all_medals = oo.groupby(['Athlete','Medal']).size().unstack('Medal', fill_value=0)
# all_medals.sort_values(['Gold','Silver','Bronze'], ascending=False)[['Gold','Silver','Bronze']].head().plot(kind='bar')

### Data Visualizations

#### Heatmaps

In [ ]:
# all_medals = oo.groupby(['NOC','Medal']).size().unstack('Medal', fill_value=0)
# all_medals_sorted = all_medals.sort_values(['Gold','Silver','Bronze'], ascending=False)[['Gold','Silver','Bronze']].head(25);

In [ ]:
# t = all_medals_sorted.transpose()
# sns.heatmap(t);

#### Colormaps

In [ ]:
# all_medals = oo.groupby(['Athlete','Medal']).size().unstack('Medal', fill_value=0)
# sorted = all_medals.sort_values(['Gold','Silver','Bronze'], ascending=False)[['Gold','Silver','Bronze']].head().plot(kind='bar')

In [ ]:
# gsb = ['#dbb40c','#c5c9c7','#a87900']
# my_pal = sns.palplot(sns.color_palette(gsb))

In [ ]:
# all_medals = oo.groupby(['Athlete','Medal']).size().unstack('Medal', fill_value=0)
# all_medals = all_medals.sort_values(['Gold','Silver','Bronze'], ascending=False)[['Gold','Silver','Bronze']].head()
# all_medals.plot(kind='bar', colormap=my_pal);

## Final Challenge

#### Problem

#### in every Olympics, which US athlete has won the most total number of medals? Include athlete's discipline.

In [ ]:
# usam = oo[oo['NOC'] == 'USA']
# usam = usam.groupby(['Edition','Athlete','Medal']).size().unstack('Medal',fill_value=0)
# usam['Total'] = usam['Gold'] + usam['Silver'] + usam['Bronze']
# # usam.reset_index(inplace=True)
# tu = [group.sort_values('Total',ascending=False)[:1] for year,group in usam.groupby('Edition')]
# med = pd.DataFrame()
# for i in tu:
#     med = med.append(i)
# med     